In [3]:
%pip install "zenml[server]" --user
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2 --user


  Using cached zenml-0.55.2-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
  Using cached azure_mgmt_resource-23.0.1-py3-none-any.whl.metadata (35 kB)
  Using cached bcrypt-4.0.1-cp36-abi3-win_amd64.whl (152 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached click_params-0.3.0-py3-none-any.whl (12 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached docker-6.1.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached fastapi-0.99.1-py3-none-any.whl.metadata (23 kB)
  Using cached fastapi_utils-0.2.1-py3-none-any.whl (16 kB)
  Using cached GitPython-3.1.42-py3-none-any.whl.metadata (12 kB)
  Using cached httplib2-0.19.1-py3-none-any.whl (95 kB)
  Using cached ipinfo-5.0.1-py3-none-any.whl.metadata (648 bytes)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached SQLAlchemy_Utils-0.38.3-py3-none-any.whl (100 kB)
  U

In [4]:
!rm -rf .zen
!zenml init

'rm' is not recognized as an internal or external command,
operable program or batch file.


NumExpr defaulting to 8 threads.
Found existing ZenML repository at path 
'c:\Users\DELL-7373\Desktop\application\MLops\notebooks'.
⠋ Initializing ZenML repository at 
c:\Users\DELL-7373\Desktop\application\MLops\notebooks.

⠋ Initializing ZenML repository at 
c:\Users\DELL-7373\Desktop\application\MLops\notebooks.




### Example Experimentation ML Code

Let us get started with some simple exemplary ML code. In the following, we train a Scikit-learn SVC classifier to classify images of handwritten digits. We load the data, train a model on the training set, then test it on the test set.

In [5]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test()-> None:
    """Train and test a Scikit-learn SVC classifier on the digits dataset."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train,X_test,y_train,y_test = train_test_split(
        data,digits.target,test_size = 0.2, shuffle=False
    )
    model=SVC(gamma=0.001)
    model.fit(X_train,y_train)
    test_acc=model.score(X_test,y_test)
    print(f"Test accuracy: {test_acc}")

train_test()
    

Test accuracy: 0.9583333333333334


### Turning experiments into ML pipelines with ZenML

In [6]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer()-> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"]
]:
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images),-1))
    X_train,X_test,y_train,y_test = train_test_split(
        data,digits.target,test_size=0.2,shuffle=False
    )
    return X_train,X_test,y_train,y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,

) -> ClassifierMixin:
    """Train an sklearn SVC classifier."""
    model = SVC(gamma=0.001)
    model.fit(X_train,y_train)
    return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test,y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc

NumExpr defaulting to 8 threads.


Similarly, we can use ZenML's `@pipeline` decorator to connect all our steps into an ML pipeline.

Note that the pipelone defination does not depend on the concrete step functions we defined above; it merely establishes a recipe for how data moves through the steps. This means we can replace steps as we wish, e.g. to run the same pipeline with different models to compare their performances.

In [7]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    """Link all the steps together in a pipeline"""
    X_train,X_test,y_train,y_test = importer()
    model = svc_trainer(X_train,y_train)
    evaluator(X_test,y_test,model)

### Running ZenML Pipeline

In [8]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


Creating database tables
Creating default workspace 'default' ...
Creating default stack in workspace default...
Creating default user 'default' ...
Setting the global active workspace to 'default'.
Setting the global active stack to default.
The current repo active workspace is no longer available. Resetting the active workspace to 'default'.
The current repo active stack is no longer available. Resetting the active stack to default.
Reloading configuration file C:\Users\DELL-7373\Desktop\application\MLops\notebooks\.zen\config.yaml
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Step importer has started.
Step importer has finished in 16.920s.
Step svc_trainer has started.
Step svc_trainer has finished in 1.562s.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 2.905s.
Pipeline run has finished in 22.527s.
You can visualize your pipeline run

In [10]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"ZenML dashboard is available at {public_url}")
        !zenml up --blocking --port {port}

    else:
        !zenml up --blocking --port {port}

start_zenml_dashboard()

^C
